In [1]:
%load_ext pyinstrument

In [2]:
import numpy as np

INT = np.s_[1:-1, 1:-1]
XINT = np.s_[2:, 1:-1]
XMINT = np.s_[:-2, 1:-1]
YINT = np.s_[1:-1, 2:]
YMINT = np.s_[1:-1, :-2]


def solve_poisson_jacobi(phi, rho, h, iters):
    # rho_(i,j) * h²
    rhs = rho[INT] * (h*h) 
    phi_next = np.zeros_like(phi)
    for i in range(iters):
        phi_next[INT] = ((phi[XINT] + phi[XMINT] + phi[YINT] + phi[YMINT]) - rhs) / 4
        phi, phi_next = phi_next, phi
            
    return phi

In [3]:

def solve_poisson_GS(phi, rho, h, iters):
    
    omega = 2 / (1 + np.sin(np.pi / phi.shape[0]))
    # rho_(i,j) * h²
    rhs = rho * (h*h) 
    for i in range(iters):
        for a in range(1, phi.shape[0] - 1):
            for b in range(1, phi.shape[1] - 1):
                phi[a, b] = (1 - omega) * phi[a, b] + omega * (((phi[a + 1, b] + phi[a - 1, b] + phi[a, b + 1] + phi[a, b - 1]) - rhs[a, b]) / 4 )
            
    return phi

In [4]:
%load_ext Cython

In [27]:
%%cython --annotate
# cython: boundscheck=False, wraparound=False, cdivision=True

import cython
import numpy as np
cimport numpy as cnp

ctypedef cnp.float64_t DTYPE_t

def solve_poisson_GS_cython(double[:, :] phi,
                            double[:, :] rho,
                            double h,
                            int iters):

    cdef:
        Py_ssize_t i, a, b
        Py_ssize_t nx = phi.shape[0] - 1
        Py_ssize_t ny = phi.shape[1] - 1
        double omega = 2.0 / (1.0 + np.sin(np.pi / phi.shape[0]))
        double h2 = h * h
        double rhs

    for i in range(iters):
        for a in range(1, nx):
            for b in range(1, ny):
                rhs = rho[a, b] * h2
                phi[a, b] = (1.0 - omega) * phi[a, b] + omega * (
                    (phi[a+1, b] + phi[a-1, b] +
                     phi[a, b+1] + phi[a, b-1] - rhs) * 0.25
                )

    return np.asarray(phi)

In [25]:
X = 257
dx = 1 / X
domain = np.linspace(0, 1, X)
x, y = np.meshgrid(domain, domain)

p_solved = np.sin(np.pi* x) * np.sin(np.pi * y) 
u = -2 * np.pi**2 * np.sin(np.pi * x ) * np.sin(np.pi * y)
p = np.random.rand(X, X)
p[0, :] = 0
p[-1, :] = 0
p[1:-1, 0] = 0
p[1:-1, -1] = 0



In [21]:
%%timeit
#%%pyinstrument
p_out = solve_poisson_jacobi(p.copy(), u, dx, 20)

4.29 ms ± 146 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
abs(p_solved - p_out).sum()

np.float64(16418.395002875593)

In [22]:
%%timeit

p_out_gs = solve_poisson_GS(p.copy(), u, dx, 12)
abs(p_solved - p_out_gs).sum()

806 ms ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
abs(p_solved - p).sum()

np.float64(22292.145057470574)

In [29]:
%%timeit
p_out_gs_cy = solve_poisson_GS_cython(p.copy(), u, dx, 20)
abs(p_solved - p_out_gs_cy).sum()

7.98 ms ± 20.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
